In [1]:
# import sys
# sys.path.append("..")
import ScraperFC as sfc # import local scraperfc
import traceback

In [2]:
# scraper = sfc.FBRef()
# year = 2020
# league = "Euros"
# try:
#     season_link = scraper.get_season_link(year=year, league=league)
#     assert season_link != -1

#     match_links = scraper.get_match_links(year=year, league=league)
#     if type(match_links) is int:
#         assert match_links != -1
#         print(match_links, season_link)
#     elif type(match_links) is list:
#         assert len(match_links) > 0
#         print(len(match_links), season_link)
#     else:
#         print(match_links, season_link)
# except:
#     traceback.print_exc()
# scraper.close()

In [3]:
# def test_sources():
scraper = sfc.FBRef()
year = 2022
league = "World Cup"
try:
    stats = scraper.scrape_stats(year=year, league=league, stat_category="standard")
except:
    traceback.print_exc()
finally:
    scraper.close()



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\oseymour\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


In [ ]:
stats

(None, None, None)